## 지번별 건물 에너지 사용량
+ 데이터명: 국토교통부 '건물에너지정보 서비스'
+ 데이터 유형: xml
+ 요청인자: 시군구코드, 법정동코드, 번, 지, 사용년월(YYYYMM)
+ 출력값: 사용량(useQty)

[공공데이터 링크](https://www.data.go.kr/data/15054212/openapi.do)

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

url = 'http://apis.data.go.kr/1611000/BldEngyService/getBeElctyUsgInfo'
params ={'serviceKey' : 'cTP7canYYa8SBQuAJzDFlO4fCg/krijsJXe3oW448oTBx8pj3b0rMaBzLpDN/AdIZim2GBbEuN1C6fIrSQ4K2w==', 'numOfRows' : '10', 'pageNo' : '1', 'sigunguCd' : '41290', 'bjdongCd' : '10700', 'bun' : '0101', 'ji' : '0000', 'useYm' : '202204' }

response = requests.get(url, params=params)
data = BeautifulSoup(response.text, 'lxml')
item = data.find_all("item")

In [ ]:
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] # 데이터값

for i in range(0, len(item)):
    columns = item[i].find_all()
    for j in range(0,len(columns)):
        if i == 0:
            name_list.append(columns[j].name)
        value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list=[]

name_list = ['법정동코드', '번', '지', '새주소본번', '새주소도로코드', '새주소부번', '도로명대지위치', '대지구분코드', '대지위치', '순번', '시군구코드', '사용량', '사용년월']

In [ ]:
print(name_list)

['bjdongcd', 'bun', 'ji', 'namainbun', 'naroadcd', 'nasubbun', 'naugrndcd', 'newplatplc', 'platgbcd', 'platplc', 'rnum', 'sigungucd', 'useqty', 'useym']


## 단지별 에너지 사용량
+ 데이터명: 국토교통부 '공동주택 에너지 사용 정보'
+ 데이터 유형: xml
+ 요청인자: 서비스키, 단지코드, 발생년월(YYYYMM)
+ 출력값: 난방사용량(금액), 급탕사용량(금액), 가스사용량(금액), 전기사용량(금액), 수도사용량(금액)

[공공데이터 링크](https://www.data.go.kr/data/15012964/openapi.do)

### 단지코드 불러오기

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os

In [ ]:
# github 연결하기
%cd '/content/drive/MyDrive/국토교통데이터활용공모전'
!git clone https://github.com/haemilia/MolitContest.git

/content/drive/MyDrive/국토교통데이터활용공모전
Cloning into 'MolitContest'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 59 (delta 7), reused 21 (delta 2), pack-reused 0
Unpacking objects: 100% (59/59), 83.56 KiB | 161.00 KiB/s, done.


In [41]:
# github의 AptList 폴더 내 자치구별 파일 리스트 불러오기
data_path = '/content/drive/MyDrive/국토교통데이터활용공모전/MolitContest/AptList'
file_list = os.listdir(data_path)

In [42]:
# 구별 단지 코드 불러오는 함수 생성
def getDanjiCode(Gu):
  data_path = '/content/drive/MyDrive/국토교통데이터활용공모전/MolitContest/AptList/'
  gu_df = pd.read_csv(data_path + Gu, encoding = 'utf-8')
  danji_codes = gu_df['kaptCode']
  return danji_codes

### 단지별 난방, 급탕, 가스, 전기, 수도 사용량 불러오기

+ 우리가 비교하고 싶은 기간이 언제부터 언제까지인지 결정

In [43]:
API_key = 'cTP7canYYa8SBQuAJzDFlO4fCg/krijsJXe3oW448oTBx8pj3b0rMaBzLpDN/AdIZim2GBbEuN1C6fIrSQ4K2w=='
url = 'http://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService/getHsmpApHusUsgQtyInfoSearch'

params ={'serviceKey' : API_key, 
              'kaptCode' : "A11041001", 
              'reqDate' : "202201"}

response = requests.get(url, params)
data = BeautifulSoup(response.text, "xml")
data.find('hwaterCool').get_text()

'5647'

In [44]:
def getEnergyUse(danji_codes, date):
  
  API_key = 'cTP7canYYa8SBQuAJzDFlO4fCg/krijsJXe3oW448oTBx8pj3b0rMaBzLpDN/AdIZim2GBbEuN1C6fIrSQ4K2w=='
  url = 'http://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService/getHsmpApHusUsgQtyInfoSearch'

  energyUse_list = []
  
  for code in danji_codes:
    params ={'serviceKey' : API_key, 
              'kaptCode' : code, 
              'reqDate' : date}
    
    response = requests.get(url, params)
    data = BeautifulSoup(response.text, "xml")

    if(data.find('helect') != None):
      if(data.find('helect').get_text() == "0"):
        continue
      else:
        elect = data.find('helect').get_text()
    else:
      continue


    if(data.find('hgas')!=None):
        gas = data.find('hgas').get_text()
    else: gas = np.nan

    if(data.find('hheat')!=None):
        heat = data.find('hheat').get_text()
    else:
        heat=np.nan

    if(data.find('hwaterCool')!=None):
        watercool = data.find('hwaterCool').get_text()
    else:
        watercool = np.nan
    
    if(data.find('hwaterHot')!=None):
        waterhot = data.find('hwaterHot').get_text()
    else:
        waterhot = np.nan
    
    data = {
        'code': code,
        'date': date,
        'elect': elect,
        'gas': gas,
        'heat': heat,
        'watercool': watercool,
        'waterhot': waterhot
    }

    energyUse_list.append(data)
  
  energy_df = pd.DataFrame(energyUse_list)

  return energy_df

### 3. 특정 기간 내 단지별 개별 에너지 사용량 csv로 저장


In [45]:
# 특정 기간 내 date list(YYYYMM) 생성
import datetime

start_date = datetime.date(2022, 1, 1)
end_date = datetime.date(2023, 4, 30)

date_list = []
current_date = start_date

while current_date <= end_date:
    date_string = current_date.strftime("%Y%m")
    date_list.append(date_string)
    current_date = current_date.replace(day=1) + datetime.timedelta(days=32)
    current_date = current_date.replace(day=1)

print(date_list)

['202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304']


In [46]:
# getDanjiCode로 특정 자치구의 단지 코드 불러오기
# for문과 getEnergyUse로 기간별 단지별 에너지 사용량 불러오기
def getGuEnergyDF(Gu_file):
  danji_codes = getDanjiCode(Gu_file)

  energy_df = pd.DataFrame()
  for date in date_list:   
    df = getEnergyUse(danji_codes, date)
    energy_df = pd.concat([energy_df, df], ignore_index=True)

  energy_df.to_csv(f'/content/drive/MyDrive/국토교통데이터활용공모전/MolitContest/EnergyUseInfo/energy_{Gu_file}', index=False)

In [53]:
for file in file_list[20:]:
  getGuEnergyDF(file)

In [ ]:
len(file_list)

24